# Pytorch for Boston Classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import (
    Dataset, DataLoader
)

from sklearn.model_selection import train_test_split

from eleven.random import random_seeds
from eleven.data import create_boston_classification

In [2]:
# Some convenient formatting
torch.set_printoptions(precision=2, sci_mode=True)

In [3]:
device = torch.device('cpu')

In [4]:
x, y = create_boston_classification()

In [5]:
x = torch.tensor(x.values, dtype=torch.float32)
y = torch.tensor(y.values)

In [6]:
# Scikit-learn can handle any array like objects, including torch tensors
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42
)

## Creating  a Pytorch Dataset

In [7]:
class Boston(Dataset):
    """Boston Classification
    
    Pytorch datasets require two methods:
    `__len__`, returns the number of samples 
    in the dataset, and `__getitem__`, which
    returns the next sample of data given by an 
    index `idx`.
    
    Args:
        x: features
        y: labels
    """
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Boston(x={self.x.shape}, y={self.y.shape})"
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [8]:
train_data = Boston(x_train, y_train)
valid_data = Boston(x_test, y_test)

In [9]:
print(f"Train Dataset: {train_data}")
print(f"Valid Dataset: {valid_data}")

Train Dataset: Boston(x=torch.Size([339, 13]), y=torch.Size([339]))
Valid Dataset: Boston(x=torch.Size([167, 13]), y=torch.Size([167]))


## A Linear Model, Multi-Class Logistic Regression

In [10]:
class LinearModel(nn.Module):
    
    def __init__(self, input_dim, num_classes):
        super(LinearModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x

In [11]:
# Our features have 13 dimensions. Our model will 
# learn weigths for each of those features. 
model = LinearModel(13, 3)

## DataLoaders

In [12]:
train_loader = DataLoader(train_data, batch_size=1)
valid_loader = DataLoader(valid_data, batch_size=1)

### DataLoader Usage:

We iterate over the dataloader, getting batches of size `batch_size`, and hand that to our model, getting softmax scores of size `(batch_size, num_classes)`.

In [13]:
for idx, (data, target) in enumerate(train_loader):
    softmax_scores = model(data)
    print(f"Model Output Shape: {softmax_scores.shape}")
    if idx == 0:
        break

Model Output Shape: torch.Size([1, 3])


### Repeatable Minibatches

If you would like have reproducible minibatches, you can use the `random_seeds` context manager I added in `eleven.random`:

In [14]:
with random_seeds():
    for idx, (data, target) in enumerate(train_loader):
        print(f"Softmax Scores: {model(data)}")
        if idx == 4:
            break

Softmax Scores: tensor([[0.00e+00, -1.29e+02, -1.55e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -1.42e+02, -7.96e+01]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -1.45e+02, -7.59e+01]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -1.50e+02, -1.72e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[0.00e+00, -1.47e+02, -9.81e+01]], grad_fn=<LogSoftmaxBackward>)


## Training Loop

In [15]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [16]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
train_data = Boston(x_train, y_train)
valid_data = Boston(x_test, y_test)

train_loader = DataLoader(train_data, batch_size=32)
valid_loader = DataLoader(valid_data, batch_size=1)

linear_model = LinearModel(13, 3)
optimizer = optim.Adam(linear_model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)


with random_seeds():
    for epoch in range(500):
        train(linear_model, device, train_loader, optimizer, epoch)
        test(linear_model, device, valid_loader)

Train Epoch: 0 [0/339 (0%)]	Loss: 116.004227
Test set: Average loss: 94.0044, Accuracy: 37/167 (22%)

Train Epoch: 1 [0/339 (0%)]	Loss: 100.684486
Test set: Average loss: 79.3981, Accuracy: 37/167 (22%)

Train Epoch: 2 [0/339 (0%)]	Loss: 85.352783
Test set: Average loss: 64.7922, Accuracy: 37/167 (22%)

Train Epoch: 3 [0/339 (0%)]	Loss: 70.020912
Test set: Average loss: 50.2138, Accuracy: 36/167 (22%)

Train Epoch: 4 [0/339 (0%)]	Loss: 54.690903
Test set: Average loss: 36.7250, Accuracy: 30/167 (18%)

Train Epoch: 5 [0/339 (0%)]	Loss: 39.955349
Test set: Average loss: 26.0960, Accuracy: 39/167 (23%)

Train Epoch: 6 [0/339 (0%)]	Loss: 26.889124
Test set: Average loss: 21.4169, Accuracy: 44/167 (26%)

Train Epoch: 7 [0/339 (0%)]	Loss: 18.390266
Test set: Average loss: 19.9212, Accuracy: 38/167 (23%)

Train Epoch: 8 [0/339 (0%)]	Loss: 17.055670
Test set: Average loss: 18.5720, Accuracy: 35/167 (21%)

Train Epoch: 9 [0/339 (0%)]	Loss: 17.095100
Test set: Average loss: 17.3575, Accuracy: 26

Test set: Average loss: 0.7301, Accuracy: 111/167 (66%)

Train Epoch: 82 [0/339 (0%)]	Loss: 0.681457
Test set: Average loss: 0.7256, Accuracy: 111/167 (66%)

Train Epoch: 83 [0/339 (0%)]	Loss: 0.673041
Test set: Average loss: 0.7212, Accuracy: 111/167 (66%)

Train Epoch: 84 [0/339 (0%)]	Loss: 0.664991
Test set: Average loss: 0.7171, Accuracy: 112/167 (67%)

Train Epoch: 85 [0/339 (0%)]	Loss: 0.657295
Test set: Average loss: 0.7131, Accuracy: 113/167 (68%)

Train Epoch: 86 [0/339 (0%)]	Loss: 0.649931
Test set: Average loss: 0.7093, Accuracy: 113/167 (68%)

Train Epoch: 87 [0/339 (0%)]	Loss: 0.642882
Test set: Average loss: 0.7057, Accuracy: 114/167 (68%)

Train Epoch: 88 [0/339 (0%)]	Loss: 0.636136
Test set: Average loss: 0.7022, Accuracy: 114/167 (68%)

Train Epoch: 89 [0/339 (0%)]	Loss: 0.629674
Test set: Average loss: 0.6989, Accuracy: 114/167 (68%)

Train Epoch: 90 [0/339 (0%)]	Loss: 0.623484
Test set: Average loss: 0.6957, Accuracy: 115/167 (69%)

Train Epoch: 91 [0/339 (0%)]	Loss:

Test set: Average loss: 0.6202, Accuracy: 128/167 (77%)

Train Epoch: 164 [0/339 (0%)]	Loss: 0.464973
Test set: Average loss: 0.6200, Accuracy: 128/167 (77%)

Train Epoch: 165 [0/339 (0%)]	Loss: 0.464327
Test set: Average loss: 0.6198, Accuracy: 128/167 (77%)

Train Epoch: 166 [0/339 (0%)]	Loss: 0.463694
Test set: Average loss: 0.6196, Accuracy: 128/167 (77%)

Train Epoch: 167 [0/339 (0%)]	Loss: 0.463074
Test set: Average loss: 0.6194, Accuracy: 128/167 (77%)

Train Epoch: 168 [0/339 (0%)]	Loss: 0.462466
Test set: Average loss: 0.6192, Accuracy: 128/167 (77%)

Train Epoch: 169 [0/339 (0%)]	Loss: 0.461871
Test set: Average loss: 0.6190, Accuracy: 128/167 (77%)

Train Epoch: 170 [0/339 (0%)]	Loss: 0.461287
Test set: Average loss: 0.6189, Accuracy: 128/167 (77%)

Train Epoch: 171 [0/339 (0%)]	Loss: 0.460714
Test set: Average loss: 0.6187, Accuracy: 128/167 (77%)

Train Epoch: 172 [0/339 (0%)]	Loss: 0.460153
Test set: Average loss: 0.6185, Accuracy: 128/167 (77%)

Train Epoch: 173 [0/339 (

Test set: Average loss: 0.6154, Accuracy: 129/167 (77%)

Train Epoch: 247 [0/339 (0%)]	Loss: 0.435289
Test set: Average loss: 0.6154, Accuracy: 129/167 (77%)

Train Epoch: 248 [0/339 (0%)]	Loss: 0.435078
Test set: Average loss: 0.6154, Accuracy: 129/167 (77%)

Train Epoch: 249 [0/339 (0%)]	Loss: 0.434868
Test set: Average loss: 0.6154, Accuracy: 129/167 (77%)

Train Epoch: 250 [0/339 (0%)]	Loss: 0.434659
Test set: Average loss: 0.6154, Accuracy: 128/167 (77%)

Train Epoch: 251 [0/339 (0%)]	Loss: 0.434452
Test set: Average loss: 0.6154, Accuracy: 128/167 (77%)

Train Epoch: 252 [0/339 (0%)]	Loss: 0.434247
Test set: Average loss: 0.6154, Accuracy: 128/167 (77%)

Train Epoch: 253 [0/339 (0%)]	Loss: 0.434043
Test set: Average loss: 0.6155, Accuracy: 128/167 (77%)

Train Epoch: 254 [0/339 (0%)]	Loss: 0.433839
Test set: Average loss: 0.6155, Accuracy: 128/167 (77%)

Train Epoch: 255 [0/339 (0%)]	Loss: 0.433639
Test set: Average loss: 0.6155, Accuracy: 128/167 (77%)

Train Epoch: 256 [0/339 (

Test set: Average loss: 0.6163, Accuracy: 128/167 (77%)

Train Epoch: 329 [0/339 (0%)]	Loss: 0.421333
Test set: Average loss: 0.6163, Accuracy: 128/167 (77%)

Train Epoch: 330 [0/339 (0%)]	Loss: 0.421191
Test set: Average loss: 0.6163, Accuracy: 128/167 (77%)

Train Epoch: 331 [0/339 (0%)]	Loss: 0.421050
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 332 [0/339 (0%)]	Loss: 0.420909
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 333 [0/339 (0%)]	Loss: 0.420769
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 334 [0/339 (0%)]	Loss: 0.420630
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 335 [0/339 (0%)]	Loss: 0.420490
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 336 [0/339 (0%)]	Loss: 0.420352
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 337 [0/339 (0%)]	Loss: 0.420212
Test set: Average loss: 0.6164, Accuracy: 128/167 (77%)

Train Epoch: 338 [0/339 (

Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 416 [0/339 (0%)]	Loss: 0.410545
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 417 [0/339 (0%)]	Loss: 0.410436
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 418 [0/339 (0%)]	Loss: 0.410328
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 419 [0/339 (0%)]	Loss: 0.410221
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 420 [0/339 (0%)]	Loss: 0.410114
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 421 [0/339 (0%)]	Loss: 0.410007
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 422 [0/339 (0%)]	Loss: 0.409900
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 423 [0/339 (0%)]	Loss: 0.409794
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 424 [0/339 (0%)]	Loss: 0.409687
Test set: Average loss: 0.6162, Accuracy: 131/167 (78%)

Train Epoch: 425 [0/339 (

In [20]:
class MLP(nn.Module):
    
    def __init__(self, input_dim, hid_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hid_dim)
        self.fc2 = nn.Linear(hid_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [29]:
mlp = MLP(13, 10, 3)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)


with random_seeds():
    for epoch in range(500):
        train(mlp, device, train_loader, optimizer, epoch)
        test(mlp, device, valid_loader)

Train Epoch: 0 [0/339 (0%)]	Loss: 56.739006
Test set: Average loss: 34.0999, Accuracy: 64/167 (38%)

Train Epoch: 1 [0/339 (0%)]	Loss: 38.917759
Test set: Average loss: 21.4798, Accuracy: 57/167 (34%)

Train Epoch: 2 [0/339 (0%)]	Loss: 24.334997
Test set: Average loss: 16.1400, Accuracy: 55/167 (33%)

Train Epoch: 3 [0/339 (0%)]	Loss: 18.026031
Test set: Average loss: 11.1404, Accuracy: 58/167 (35%)

Train Epoch: 4 [0/339 (0%)]	Loss: 12.500512
Test set: Average loss: 5.7628, Accuracy: 53/167 (32%)

Train Epoch: 5 [0/339 (0%)]	Loss: 5.544951
Test set: Average loss: 4.2681, Accuracy: 35/167 (21%)

Train Epoch: 6 [0/339 (0%)]	Loss: 3.533892
Test set: Average loss: 2.7534, Accuracy: 66/167 (40%)

Train Epoch: 7 [0/339 (0%)]	Loss: 2.502891
Test set: Average loss: 1.8204, Accuracy: 78/167 (47%)

Train Epoch: 8 [0/339 (0%)]	Loss: 1.635023
Test set: Average loss: 1.4676, Accuracy: 86/167 (51%)

Train Epoch: 9 [0/339 (0%)]	Loss: 1.429811
Test set: Average loss: 1.1819, Accuracy: 86/167 (51%)

T

Test set: Average loss: 0.6664, Accuracy: 119/167 (71%)

Train Epoch: 89 [0/339 (0%)]	Loss: 0.511312
Test set: Average loss: 0.6660, Accuracy: 120/167 (72%)

Train Epoch: 90 [0/339 (0%)]	Loss: 0.510340
Test set: Average loss: 0.6656, Accuracy: 121/167 (72%)

Train Epoch: 91 [0/339 (0%)]	Loss: 0.509367
Test set: Average loss: 0.6652, Accuracy: 122/167 (73%)

Train Epoch: 92 [0/339 (0%)]	Loss: 0.508508
Test set: Average loss: 0.6648, Accuracy: 121/167 (72%)

Train Epoch: 93 [0/339 (0%)]	Loss: 0.507492
Test set: Average loss: 0.6645, Accuracy: 121/167 (72%)

Train Epoch: 94 [0/339 (0%)]	Loss: 0.506557
Test set: Average loss: 0.6641, Accuracy: 121/167 (72%)

Train Epoch: 95 [0/339 (0%)]	Loss: 0.505679
Test set: Average loss: 0.6638, Accuracy: 121/167 (72%)

Train Epoch: 96 [0/339 (0%)]	Loss: 0.504786
Test set: Average loss: 0.6635, Accuracy: 123/167 (74%)

Train Epoch: 97 [0/339 (0%)]	Loss: 0.503904
Test set: Average loss: 0.6631, Accuracy: 123/167 (74%)

Train Epoch: 98 [0/339 (0%)]	Loss:

Test set: Average loss: 0.6498, Accuracy: 124/167 (74%)

Train Epoch: 170 [0/339 (0%)]	Loss: 0.469148
Test set: Average loss: 0.6496, Accuracy: 124/167 (74%)

Train Epoch: 171 [0/339 (0%)]	Loss: 0.468880
Test set: Average loss: 0.6495, Accuracy: 123/167 (74%)

Train Epoch: 172 [0/339 (0%)]	Loss: 0.468619
Test set: Average loss: 0.6494, Accuracy: 123/167 (74%)

Train Epoch: 173 [0/339 (0%)]	Loss: 0.468511
Test set: Average loss: 0.6492, Accuracy: 123/167 (74%)

Train Epoch: 174 [0/339 (0%)]	Loss: 0.468166
Test set: Average loss: 0.6491, Accuracy: 123/167 (74%)

Train Epoch: 175 [0/339 (0%)]	Loss: 0.467867
Test set: Average loss: 0.6490, Accuracy: 123/167 (74%)

Train Epoch: 176 [0/339 (0%)]	Loss: 0.467646
Test set: Average loss: 0.6489, Accuracy: 123/167 (74%)

Train Epoch: 177 [0/339 (0%)]	Loss: 0.467419
Test set: Average loss: 0.6488, Accuracy: 123/167 (74%)

Train Epoch: 178 [0/339 (0%)]	Loss: 0.467172
Test set: Average loss: 0.6487, Accuracy: 123/167 (74%)

Train Epoch: 179 [0/339 (

Test set: Average loss: 0.6466, Accuracy: 128/167 (77%)

Train Epoch: 258 [0/339 (0%)]	Loss: 0.455140
Test set: Average loss: 0.6465, Accuracy: 128/167 (77%)

Train Epoch: 259 [0/339 (0%)]	Loss: 0.454981
Test set: Average loss: 0.6464, Accuracy: 128/167 (77%)

Train Epoch: 260 [0/339 (0%)]	Loss: 0.454823
Test set: Average loss: 0.6463, Accuracy: 128/167 (77%)

Train Epoch: 261 [0/339 (0%)]	Loss: 0.454665
Test set: Average loss: 0.6462, Accuracy: 128/167 (77%)

Train Epoch: 262 [0/339 (0%)]	Loss: 0.454509
Test set: Average loss: 0.6461, Accuracy: 128/167 (77%)

Train Epoch: 263 [0/339 (0%)]	Loss: 0.454353
Test set: Average loss: 0.6460, Accuracy: 128/167 (77%)

Train Epoch: 264 [0/339 (0%)]	Loss: 0.454197
Test set: Average loss: 0.6460, Accuracy: 128/167 (77%)

Train Epoch: 265 [0/339 (0%)]	Loss: 0.454042
Test set: Average loss: 0.6459, Accuracy: 128/167 (77%)

Train Epoch: 266 [0/339 (0%)]	Loss: 0.453888
Test set: Average loss: 0.6458, Accuracy: 128/167 (77%)

Train Epoch: 267 [0/339 (

Train Epoch: 344 [0/339 (0%)]	Loss: 0.441780
Test set: Average loss: 0.6403, Accuracy: 129/167 (77%)

Train Epoch: 345 [0/339 (0%)]	Loss: 0.441726
Test set: Average loss: 0.6402, Accuracy: 129/167 (77%)

Train Epoch: 346 [0/339 (0%)]	Loss: 0.441510
Test set: Average loss: 0.6401, Accuracy: 129/167 (77%)

Train Epoch: 347 [0/339 (0%)]	Loss: 0.441463
Test set: Average loss: 0.6401, Accuracy: 129/167 (77%)

Train Epoch: 348 [0/339 (0%)]	Loss: 0.441194
Test set: Average loss: 0.6401, Accuracy: 130/167 (78%)

Train Epoch: 349 [0/339 (0%)]	Loss: 0.441155
Test set: Average loss: 0.6400, Accuracy: 129/167 (77%)

Train Epoch: 350 [0/339 (0%)]	Loss: 0.440894
Test set: Average loss: 0.6399, Accuracy: 130/167 (78%)

Train Epoch: 351 [0/339 (0%)]	Loss: 0.440930
Test set: Average loss: 0.6398, Accuracy: 130/167 (78%)

Train Epoch: 352 [0/339 (0%)]	Loss: 0.440676
Test set: Average loss: 0.6397, Accuracy: 130/167 (78%)

Train Epoch: 353 [0/339 (0%)]	Loss: 0.440621
Test set: Average loss: 0.6396, Accur

Test set: Average loss: 0.6353, Accuracy: 130/167 (78%)

Train Epoch: 425 [0/339 (0%)]	Loss: 0.430779
Test set: Average loss: 0.6353, Accuracy: 130/167 (78%)

Train Epoch: 426 [0/339 (0%)]	Loss: 0.430647
Test set: Average loss: 0.6352, Accuracy: 130/167 (78%)

Train Epoch: 427 [0/339 (0%)]	Loss: 0.430514
Test set: Average loss: 0.6352, Accuracy: 130/167 (78%)

Train Epoch: 428 [0/339 (0%)]	Loss: 0.430381
Test set: Average loss: 0.6351, Accuracy: 130/167 (78%)

Train Epoch: 429 [0/339 (0%)]	Loss: 0.430249
Test set: Average loss: 0.6351, Accuracy: 130/167 (78%)

Train Epoch: 430 [0/339 (0%)]	Loss: 0.430116
Test set: Average loss: 0.6350, Accuracy: 130/167 (78%)

Train Epoch: 431 [0/339 (0%)]	Loss: 0.429984
Test set: Average loss: 0.6350, Accuracy: 130/167 (78%)

Train Epoch: 432 [0/339 (0%)]	Loss: 0.429809
Test set: Average loss: 0.6349, Accuracy: 130/167 (78%)

Train Epoch: 433 [0/339 (0%)]	Loss: 0.429760
Test set: Average loss: 0.6348, Accuracy: 130/167 (78%)

Train Epoch: 434 [0/339 (